In [1]:
import pandas as pd
import numpy as np
# from sklearn.mixture import GaussianMixture
import multiprocessing
from multiprocessing.pool import MapResult
from Utils import remove_splchars,clean
from pyarrow.parquet import ParquetFile
import pyarrow as pa

# Preprocessing

In [2]:
data = ParquetFile("paraquet/train.parquet")
batch = next(data.iter_batches(batch_size = 1000)) 
batch = pa.Table.from_batches([batch]).to_pandas()
batch.drop(["id"], axis=1, inplace=True)

In [ ]:
df = pd.read_parquet("paraquet/train.parquet", engine="fastparquet")
df.drop(["id"], axis=1, inplace=True)
print(len(df))

In [11]:
def split(df, NUM_SPLITS: int =6):
    splits = ()
    split_range = len(df)//NUM_SPLITS
    for ind in range(NUM_SPLITS):
        splits+= (df.iloc[ind:ind+split_range,...],)
    return splits


cur_data = split(batch)

In [3]:
#pool = multiprocessing.Pool()
with multiprocessing.Pool(5) as p:
    batch["article"] = p.map(remove_splchars,batch["article"])

In [4]:
batch.loc[0,"article"]

" 1411 EST, 25 October 2013 . 1536 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda pictured of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through contaminated food while attending a conference for newly ordained bishops in Italy last month. Symptoms of hepatitis A in

In [5]:
from train_util import phi_2

phi_2.train_start(batch)


: 

In [10]:
len(batch)

1000